In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.integrate import simps
import matplotlib.ticker as ticker

In [ ]:
import numpy as np
from scipy import interpolate


def dvr(r, e, mu, interp=False):
    """
    Discrete Value Representation (DVR) 1-Dimensional

        r   : (array) potential surface position values
        e   : (array) potential surface energy values
        mu  : (float) reduce mass
    """

    if interp:
        cs = interpolate.CubicSpline(r, e)
        r_new = np.linspace(r[0], r[-1], 500)
        e_new = cs(r_new)

        r = r_new
        e = e_new

    N = len(r)
    mp = (r[1] - r[0])

    H = np.ndarray((N,N), dtype="float")

    for i in range(N):
        for j in range(N):
            if(i == j):
                H[i][j] = (1 / (2 * mu * mp**2)) * (np.pi**2 / 3) + e[i]

            else:
                H[i][j] = (-1)**(i - j) * (1 / (2 * mu * mp**2)) * (2/ (i-j)**2)

    Ev, eigvec = np.linalg.eigh(H)
    Cv = eigvec.transpose()
    
    return [r, e, Cv, Ev]

In [ ]:
data_0 = np.loadtxt('test/n2_gs.txt', delimiter=',')
data_g = np.loadtxt('test/n2_ex_g.txt', delimiter=',')
data_u = np.loadtxt('test/n2_ex_u.txt', delimiter=',')

r_0 = data_0[:, 0] * 1.88973
e_0 = data_0[:, 1]

r_g = data_g[:, 0] * 1.88973
e_g = data_g[:, 1]

r_u = data_u[:, 0] * 1.88973
e_u = data_u[:, 1]

mu = (25526.042373971035 * 25526.042373971035) / (2*25526.042373971035)

r_0, e_0, Cv_0, Ev_0 = dvr(r_0, e_0, mu, interp=True)
r_g, e_g, Cv_g, Ev_g = dvr(r_g, e_g, mu, interp=True)
r_u, e_u, Cv_u, Ev_u = dvr(r_u, e_u, mu, interp=True)

In [ ]:
plt.rcParams.update({'font.size': 6})
plt.rcParams['axes.linewidth'] = 0.4
plt.rcParams['xtick.major.size'] = 2
plt.rcParams['xtick.major.width'] = 0.4
plt.rcParams['ytick.major.size'] = 2
plt.rcParams['ytick.major.width'] = 0.4
plt.rcParams['ytick.minor.size'] = 1
plt.rcParams['ytick.minor.width'] = 0.2

n = 1

sf = 0.1
shift_plot = -15

fig = plt.figure(figsize=(2.5, 4), dpi=300)
gs = GridSpec(1, 1)

###################################################################################
#----------------------------------------------------------------------------------
# Plot 1: Ground State
#----------------------------------------------------------------------------------
###################################################################################

ax1 = fig.add_subplot(gs[0])
ax1.plot(r_0 / 1.88973, e_0, linewidth=0.4, color='k')
ax1.plot(r_0 / 1.88973, sf * Cv_0[0] + Ev_0[0], linewidth=0.4, color='cornflowerblue')

ax1.plot(r_g / 1.88973, e_g + shift_plot, linewidth=0.4, color='orange')
ax1.plot(r_g / 1.88973, sf * Cv_g[n] + Ev_g[n] + shift_plot, linewidth=0.4, color='cornflowerblue')

ax1.plot(r_u / 1.88973, e_u + shift_plot, linewidth=0.4, color='green')
ax1.plot(r_u / 1.88973, sf * Cv_u[n] + Ev_u[n] + shift_plot, linewidth=0.4, color='cornflowerblue')


ax1.vlines(r_g[np.argmin(e_0)] / 1.88973, np.min(e_0) - 0.01, np.min(e_0) + 0.01, colors='r', linewidths=0.4)
ax1.vlines(r_g[np.argmin(e_g)] / 1.88973, np.min(e_0) - 0.01, np.min(e_0) + 0.06, colors='r', linewidths=0.4)

# Axis Labels and Limits
#----------------------------------------------------------------------------------

fig.align_ylabels()
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
def moorse(r, D=1, a=1, re=1, y=0):
    y = D * (1-np.exp(-a * (r-re) ))**2 + y
    return y

In [ ]:
De_g = 67887.4285714286 * (1 / 219474.63)
re_g = r_g[np.argmin(e_g)]
a_g = 1.6

r_mor_start = r_g[0]
r_mor_end = 5

r_mor = np.linspace(r_mor_start, r_mor_end, 500)
e_mor = moorse(r_mor, D=De_g, a=a_g, re=re_g, y=np.min(e_g))

r_mor, e_mor, Cv_mor, Ev_mor = dvr(r_mor, e_mor, mu, interp=False)

In [ ]:
plt.plot(r_g, e_g)
plt.plot(r_mor, e_mor)

In [ ]:
plt.plot(r_0 / 1.88973, Cv_0[0], linewidth=0.4, color='k')
plt.plot(r_0 / 1.88973, Cv_0[1], linewidth=0.4, color='gray')
plt.plot(r_g / 1.88973, Cv_g[0], linewidth=0.4, color='cornflowerblue')
plt.plot(r_g / 1.88973, Cv_g[1], linewidth=0.4, color='orange')

plt.plot(r_g / 1.88973, Cv_0[0] * Cv_g[0], linewidth=0.4, color='cornflowerblue', linestyle="--")
plt.plot(r_g / 1.88973, Cv_0[0] * Cv_g[1], linewidth=0.4, color='orange', linestyle="--")
plt.plot(r_g / 1.88973, Cv_0[0] * Cv_0[1], linewidth=0.4, color='gray', linestyle="--")

plt.hlines(0, 0, 2, color='r')
plt.vlines(r_g[np.argmax(Cv_0[0])] / 1.88973, -0.18, 0.18, color='r')

plt.xlim(0.8, 1.4)

In [ ]:
def franck_condon_factors(r, wfn_0, wfn_v):
    """ 
    Compute Franck-Condon factors from two vibrational wavefunction overlap

        wfn_0 : (array) vibrational wavefunction
        wfn_v : (array) vibrational wavefunction
    """
    N0 = np.sqrt(simps(wfn_0 * wfn_0, r))
    Nv = np.sqrt(simps(wfn_v * wfn_v, r))

    prod = wfn_0 / N0 * wfn_v / Nv
    fc = simps(prod, r)

    return fc

In [ ]:
S00 = franck_condon_factors(r_0, Cv_0[0], Cv_mor[0])
S01 = franck_condon_factors(r_0, Cv_0[0], Cv_mor[1])

S00u = franck_condon_factors(r_0, Cv_0[0], Cv_u[0])
S01u = franck_condon_factors(r_0, Cv_0[0], Cv_u[1])


E00 = 0
E01 = 1

In [ ]:
plt.vlines(np.array([E00, E01]) * 27.2114, [0,0], [S00, S01])
plt.hlines(1, 0, 28, color='k')

In [ ]:
plt.figure(figsize=(6, 3))

gshift = -((Ev_u[0] - Ev_0[0]) * 27.2114  - 409.9) + 0.04

plt.vlines((Ev_g[0] - Ev_0[0]) * 27.2114 + gshift, 0, S00)
plt.vlines((Ev_u[0] - Ev_0[0]) * 27.2114 + gshift, 0, S00u)

plt.vlines((Ev_g[1] - Ev_0[0]) * 27.2114 + gshift, 0, S01, colors='orange')
plt.vlines((Ev_u[1] - Ev_0[0]) * 27.2114 + gshift, 0, S01u, colors='orange')

plt.gca().xaxis.set_ticks([409.5, 409.7, 409.9, 410.1, 410.3, 410.5])

plt.xlim(409.5, 410.5)
plt.ylim(0, 1.6)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def lorentz(e, f, y=0.2, scale=1, bounds=None, offset=None, npts=10000):
    """
    Lorentzian convolution function
        e       : energy values, can be given as a number or array of numbers
        f       : intensity values, can be given as a number or array of numbers
        y       : gamma broadening factor (FWHM)
        scale   : scaling factor
        bounds  : (min, max) range for energy axis
        offset  : extension beyond bounds
        npts    : number of points on energy axis
    """

    e = np.asarray(e)
    f = np.asarray(f)

    if e.shape != f.shape:
        raise ValueError(f'e and f must have same dimension, but have shapes {e.shape} and {f.shape}')

    # make spectrum range
    s = offset or (4 * y)
    b = bounds or (e.min(), e.max())
    x = np.linspace(b[0] - s, b[1] + s, npts)

    # compute lorentzian spectrum
    spectrum = scale * f[..., np.newaxis] * (0.5 * y / np.pi) / (0.25 * y**2 + (x - e[..., np.newaxis])**2)

    return x, spectrum.squeeze()

In [35]:
a = np.array([1,2,3])
b = np.array([10,2,30])
x, s = lorentz(a, b, y=0.2)

print(x.shape)
print(s.shape)

# plt.plot(x, s.T, color='r', linewidth=0.6, linestyle='--')
# plt.plot(x, s.sum(0), color='k', linewidth=0.6)
# plt.vlines(a, 0, b, color='r', linewidth=0.6)

(10000,)
(3, 10000)


In [36]:
a = np.array([[1,2,3], [1,2,3]])
b = np.array([[10,2,30], [10,1,30]])
x, s = lorentz(a, b, y=0.2)

print(x.shape)
print(s.shape)

(10000,)
(2, 3, 10000)


In [37]:
a = np.array([1])
b = np.array([10])
x, s = lorentz(a, b, y=0.2)

print(x.shape)
print(s.shape)

(10000,)
(10000,)


In [38]:
a = np.array(1)
b = np.array(10)
x, s = lorentz(a, b, y=0.2)

print(x.shape)
print(s.shape)

TypeError: len() of unsized object

In [33]:
a = np.array([[[1,2], [1,2]], [[1,2], [1,2]]])
b = np.array([[[1,2], [1,2]], [[1,2], [1,2]]])
x, s = lorentz(a, b, y=0.2)

print(x.shape)
print(s.shape)

(10000,)
(2, 2, 2, 10000)


In [41]:
np.asarray(10).shape

()